# Django Caching Framework is what you need.

Django allows caching the following:

- Local memory cache

- Database cache

- File system cache

- Memcached

## Local Memory Caching

 Django uses a local memory caching backend. This method stores cached data in the RAM where your Django application runs. It's quick and is an excellent choice for your local development or testing environment. The disadvantage of local memory caching is its inability to scale beyond a single server. It works best with a single Django instance.

In Django, you can configure local memory caching using the built-in backend django.core.cache.backends.locmem.LocMemCache. Add the code below to your project's settings.py file to do this:

```python
CACHES = {
   'default': {
      'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
      'LOCATION': 'bookstore_cache',
   }
}
```
The cache LOCATION denotes the memory stores. If you only have one locmem cache, the LOCATION variable is not necessary. But, if you have multiple local memory caches, name at least one of them to differentiate them.

## Database Caching
Your store is growing with an increasing number of customers and books, and you wish to limit the number of database queries.

Database caching stores the cache results in your existing database. It is the simplest but not the best way to cache your Django app. It uses your app's existing database, thereby increasing the overall database load.

To configure database caching, add the following code to your project's settings.py file:

```python
CACHES = {
   'default': {
      'BACKEND': 'django.core.cache.backends.db.DatabaseCache',
      'LOCATION': 'bookstore_cache',
   }
}
```

Next, create the cache table my_table_name.

To create the cache table, execute the following command in your terminal or command prompt:
```bash
python manage.py createcachetable

## File System Cache

File system caching stores cached files in a specified file/directory location. Add the following code to your Django project's settings.py file to set up a file system caching:

```python
CACHES = {
   'default': {
      'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
      'LOCATION': '/path/to/bookstore/books/cache',
   }
}
```

File system cache benefits include easy setup, long-term data retention. It is great for storing static data or large object caches. It's no surprise that many developers prefer this method.

However, this method has its challenges. You might face performance issues with large amounts of data. Operating system restrictions could limit the number of files. In a distributed environment, sharing this cache needs additional solutions. Existing data management and security also need attention.

File system cache provides a reliable space to store data, but it may not be suitable for high-load scenarios. In such cases, Memcached can help.

## Memcached

Memcached is the most efficient and fastest caching method. Many high-traffic sites rely on Memcached to reduce the number of queries sent to the database in Django caching.

Memcached is a daemon server that handles caching. It allocates memory for caching. Add the following code to your Django project's settings.py file to set up Memcached:

```python
CACHES = {
   'default': {
      'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
      'LOCATION': '127.0.0.1:11211',
   }
}
```
Here, LOCATION refers to the address and port where your Memcached server operates.

Next, install the Memcache dependency library
```bash
pip install python-memcached
```

Memcached provides excellent performance and scalability, making it an excellent choice for caching data in large, high-load web applications.

Although Memcached offers potent capabilities for data caching and performance improvements, per-query caching may not be adequate in specific scenarios. For instance, if your bookstore application includes dynamic pages, frequently updated data, or high user personalization levels, site-wide caching becomes preferable.

# Caching the Entire Site

You can implement caching at different levels in Django. You can cache the whole site or specific parts of your website:

Per-site cache

Per-view cache

Template fragment cache

The simplest and easiest way to work with Django cache is to cache your whole website. Add two middleware classes, Update and Fetch, to your project's settings.py file to cache your entire application:
```python
MIDDLEWARE = [
   'django.middleware.cache.UpdateCacheMiddleware',     # UPDATE
   'django.middleware.common.CommonMiddleware',
   'django.middleware.cache.FetchFromCacheMiddleware',  # FETCH
]
```
The order is important, django.middleware.cache.UpdateCacheMiddleware should come before django.middleware.cache.FetchFromCacheMiddleware.

Next, add the following code to your project's settings.py file:
```python
CACHE_MIDDLEWARE_ALIAS = 'default'

CACHE_MIDDLEWARE_SECONDS = '600'
```
The CACHE_MIDDLEWARE_ALIAS variable sets the cache alias for storage, while the CACHE_MIDDLEWARE_SECONDS variable gives the number of seconds for caching a page.

## Per-View Cache

Rather than using an expensive caching level that wastes memory space, you can cache a specific view. It's an efficient way to incorporate caching in your Django app. For example, you could cache a welcome page unique to each user. Use the cache_page decorator that comes with Django either directly on the view function or the path within URLConf. It's straightforward to set up and use.

Add the following code to your app views.py file to cache your view:
```python
from django.shortcuts import HttpResponse
from django.views.decorators.cache import cache_page

@cache_page(60 * 15)
def greetings(request, first_name, last_name):
    message = "<h2>Welcome {} {} </h2>".format(first_name, last_name)
    return HttpResponse(message)
```
Next, map your views to your URLs in urls.py file:
```python
from django.urls import path
from django.views.decorators.cache import cache_page
from my_app import views

urlpatterns = [
   path('<str:first_name>/<str:last_name>', views.greetings, name='greetings'),
]
```
To cache a view in your URL, use the cache_page(60 * 15) decorator. It takes the expiry time of the cache in seconds as its argument. In the above example, the result of your greetings function will be cached for 15 minutes (60 * 15 = 900 seconds).

As the view mapped to your URL takes parameters, each different request from end users will be cached separately. For example, a request to /user/James/Brown will be different from /user/Charles/Ugo.

You can also cache a view directly in your urls.py file. Remove the cache_page(60 * 15) decorator in your greetings view and add it to the URL mapped to your greetings view:
```python
from django.urls import path
from django.views.decorators.cache import cache_page
from my_app import views

urlpatterns = [
   path('<str:first_name>/<str:last_name>', cache_page(60 * 15)(views.full_name), name='fullname')
]
```
The cache_page decorator specifies the cache expiration time in seconds.